In [1]:
import cv2
import os

data_path='Validation'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))

print(label_dict)
print(categories)
print(labels)

{'female': 0, 'male': 1}
['female', 'male']
[0, 1]


In [2]:
img_size=32
data=[]
target=[]

facedata = "haarcascade_frontalface_default.xml"
cascade = cv2.CascadeClassifier(facedata)


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        faces = cascade.detectMultiScale(img)
        try:
            for f in faces:
                x, y, w, h = [v for v in f]
                sub_face = img[y:y + h, x:x + w]
                gray=cv2.cvtColor(sub_face,cv2.COLOR_BGR2GRAY)           
                resized=cv2.resize(gray,(img_size,img_size))#some images are small and big(32 by 32)
                data.append(resized)
                target.append(label_dict[category])
        except Exception as e:#forget to use libraries
            print('Exception:',e)

In [3]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
data=np.array(data)/255.0#image converted into 0 and 1
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)#is used to convert array of labeled data(from 0 to nb_classes - 1) to one-hot vector.

np.save('./training/data',data)
np.save('./training/target',new_target)



In [4]:
import numpy as np

data=np.load('./training/data.npy')
target=np.load('./training/target.npy')

In [5]:
#create model
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint


noOfFilters=64
sizeOfFilter1=(3,3)
sizeOfFilter2=(3,3)
sizeOfPool=(2,2)#to reduce the resolution of the feature
noOfNode=64


#Conv2d is the number of filters that the convolutional layer will lear
model=Sequential()
model.add((Conv2D(32, sizeOfFilter1, input_shape=data.shape[1:],activation='relu')))#shape because we are convert data into numpy
model.add((Conv2D(32, sizeOfFilter1,activation='relu')))
model.add(MaxPooling2D(pool_size=sizeOfPool))

model.add((Conv2D(64, sizeOfFilter2,activation='relu')))
model.add((Conv2D(64, sizeOfFilter2,activation='relu')))
model.add(MaxPooling2D(pool_size=sizeOfPool))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(noOfNode, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
#flatten for converting the data into a 1-dimensional array for inputting it to the next layer
#softmax is as the activation function in the output layer of neural network models

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [6]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [7]:
checkpoint = ModelCheckpoint('./training/model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)
#checkpoint is the weights of the model. These weights can be used to make predictions as is, or used as the basis for 
#ongoing training

Epoch 1/20
92/92 [==============================] - 17s 149ms/step - loss: 0.6924 - accuracy: 0.5210 - val_loss: 0.6847 - val_accuracy: 0.5171
Epoch 2/20
92/92 [==============================] - 12s 129ms/step - loss: 0.6264 - accuracy: 0.6363 - val_loss: 0.4466 - val_accuracy: 0.8276
Epoch 3/20
92/92 [==============================] - 13s 137ms/step - loss: 0.4256 - accuracy: 0.8293 - val_loss: 0.2991 - val_accuracy: 0.8906
Epoch 4/20
92/92 [==============================] - 11s 116ms/step - loss: 0.3209 - accuracy: 0.8775 - val_loss: 0.2306 - val_accuracy: 0.9097
Epoch 5/20
92/92 [==============================] - 11s 116ms/step - loss: 0.2574 - accuracy: 0.9121 - val_loss: 0.2171 - val_accuracy: 0.9179
Epoch 6/20
92/92 [==============================] - 11s 116ms/step - loss: 0.2065 - accuracy: 0.9288 - val_loss: 0.1817 - val_accuracy: 0.9398
Epoch 7/20
92/92 [==============================] - 11s 116ms/step - loss: 0.1875 - accuracy: 0.9319 - val_loss: 0.1695 - val_accuracy: 0.9439

In [8]:
from keras.models import load_model
import cv2
import numpy as np

In [9]:
model = load_model('./training/model-007.model')

face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap=cv2.VideoCapture(0)

labels_dict={0:'Male',1:'Female'}
color_dict={0:(0,0,255),1:(0,255,0)}

In [ ]:
while(True):

    ret,img=cap.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,3)  
#detectMultiScale is function which help us to find the features/locations of the new image
    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(32,32))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,32,32,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('Result',img)
    k=cv2.waitKey(1)
    
    if k==ord("q"):
        break
        
cv2.destroyAllWindows()
cap.release()